In [2]:
import os
import time

import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'HTBot.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
from Database.models import Symbol, Candle

In [4]:
symbols = Symbol.objects.all()
len(symbols)
for symbol in symbols:
    print(symbol)

BTCUSDT
ETHUSDT
DOTUSDT
BNBUSDT
TRXUSDT
DOGEUSDT


In [5]:
# # Symbol creations
# Symbol.objects.create(symbol="DOTUSDT")
# Symbol.objects.create(symbol="BNBUSDT")
# Symbol.objects.create(symbol="TRXUSDT")
# Symbol.objects.create(symbol="DOGEUSDT")

<Symbol: DOGEUSDT>

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from ExchangeAPI.APICallManager import CandleAgent, Interval

def process_candles(symbol, interval):
    agent = CandleAgent(symbol=symbol, interval=interval)
    for i in range(10):
        print(f"Processing {symbol} at {interval}, iteration {i}")
        candles = agent.fetch_past_candles(days=20)
        agent.save_to_db(candles)
        time.sleep(1)
    return f"Completed {symbol} at {interval}"

def main():
    symbols = Symbol.objects.all()
    intervals = [Interval.MIN_15, Interval.MIN_30, Interval.HOUR_1, Interval.HOUR_4]

    tasks = [(symbol.symbol, interval) for symbol in symbols for interval in intervals]

    max_workers = min(10, len(tasks))
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_candles, symbol, interval) for symbol, interval in tasks]

        for future in as_completed(futures):
            try:
                result = future.result()
                print(result)
            except Exception as e:
                print(f"Error occurred: {e}")

if __name__ == "__main__":
    main()

ERROR! Session/line number was not unique in database. History logging moved to new session 162
Processing ETHUSDT at Interval.MIN_15, iteration 0
Processing BTCUSDT at Interval.HOUR_4, iteration 0
Processing DOTUSDT at Interval.MIN_15, iteration 0
Processing BTCUSDT at Interval.MIN_30, iteration 0
Processing ETHUSDT at Interval.MIN_30, iteration 0
Processing BTCUSDT at Interval.HOUR_1, iteration 0
Processing BTCUSDT at Interval.MIN_15, iteration 0
Processing ETHUSDT at Interval.HOUR_4, iteration 0
Processing ETHUSDT at Interval.HOUR_1, iteration 0
Processing DOTUSDT at Interval.MIN_30, iteration 0
Saved/updated 200 rows to candles table for ETHUSDT (interval: 14400000ms).
Saved/updated 200 rows to candles table for BTCUSDT (interval: 14400000ms).
Processing ETHUSDT at Interval.HOUR_4, iteration 1
Processing BTCUSDT at Interval.HOUR_4, iteration 1
Saved/updated 200 rows to candles table for ETHUSDT (interval: 14400000ms).
Saved/updated 200 rows to candles table for BTCUSDT (interval: 1

In [24]:
8+Candle.objects.filter(symbol__symbol="TRXUSDT", interval=Interval.HOUR_4.value[1]).first().open_time

1737907200000

In [9]:
def will_success(tp_percentage, sl_percentage, sign_candle: Candle, position_type: str):
    future_candles = Candle.objects.filter(
        symbol=sign_candle.symbol,
        interval=sign_candle.interval,
        open_time__gt=sign_candle.open_time
    )
    price = sign_candle.close
    if position_type == "long":
        tp_price = price * (1 + tp_percentage / 100)
        sl_price = price * (1 - sl_percentage / 100)
        for future_candle in future_candles:
            if future_candle.high >= tp_price:
                return True, future_candle.open_time
            elif future_candle.low <= sl_price:
                return False, future_candle.open_time
        return False, datetime.now().timestamp() * 1000
    else:
        sl_price = price * (1 + sl_percentage / 100)
        tp_price = price * (1 - tp_percentage / 100)
        for future_candle in future_candles:
            if future_candle.high >= sl_price:
                return False, future_candle.open_time
            elif future_candle.low <= tp_price:
                return True, future_candle.open_time
        return False, datetime.now().timestamp() * 1000

In [52]:
from django.db.models import QuerySet

# Strategy
def strategy(candles: QuerySet[Candle], tp_percentage: int, sl_percentage: int):
    total_counts = 0
    success_counts = 0
    last_time = candles.first().open_time
    for two_previous_candle, previous_candle, candle in zip(candles, candles[1:], candles[2:]):

        if candle.open_time <= last_time:
            continue

        if two_previous_candle.is_green() and previous_candle.is_green() and candle.is_green():
            total_counts += 1
            successful, last_time = will_success(
                tp_percentage=tp_percentage,
                sl_percentage=sl_percentage,
                sign_candle=candle,
                position_type="long",
            )
            if successful:
                success_counts += 1
                print("SUCCESS LONG")
                print(datetime.fromtimestamp(candle.open_time / 1000))
                print("**********************\n")
            else:
                print("FAIL LONG")
                print(datetime.fromtimestamp(candle.open_time / 1000))
                print("**********************\n")
        elif (not two_previous_candle.is_green() and
              not previous_candle.is_green() and
              not candle.is_green()):
            total_counts += 1
            successful, last_time = will_success(
                tp_percentage=tp_percentage,
                sl_percentage=sl_percentage,
                sign_candle=candle,
                position_type="short",
            )
            if successful:
                print("SHORT")
                print(datetime.fromtimestamp(candle.open_time / 1000))
                print("**********************\n")
                success_counts += 1
            else:
                print("FAIL SHORT")
                print(datetime.fromtimestamp(candle.open_time / 1000))
                print("**********************\n")

    return total_counts, success_counts

In [47]:
from datetime import datetime, timedelta
end_time = (datetime.now() - timedelta(days=0)).timestamp() * 1000
start_time = (datetime.now() - timedelta(days=30)).timestamp() * 1000


In [48]:
candles = Candle.objects.filter(symbol__symbol="TRXUSDT",
                                open_time__gte=start_time,
                                open_time__lte=end_time,
                                interval=Interval.HOUR_4.value[1])

In [49]:
print(candles.first().open_time)
print(candles.last().open_time)

1752667200000
1755172800000


In [50]:
strategy(candles=candles, tp_percentage=5, sl_percentage=1)

SUCCESS LONG
2025-07-16 20:00:00
**********************

FAIL LONG
2025-07-18 04:00:00
**********************

SHORT
2025-07-20 12:00:00
**********************

FAIL LONG
2025-07-24 16:00:00
**********************

SUCCESS LONG
2025-07-25 20:00:00
**********************

FAIL SHORT
2025-07-30 12:00:00
**********************

FAIL SHORT
2025-07-31 20:00:00
**********************

FAIL SHORT
2025-08-01 20:00:00
**********************

FAIL SHORT
2025-08-02 16:00:00
**********************

SUCCESS LONG
2025-08-04 00:00:00
**********************

SUCCESS LONG
2025-08-11 20:00:00
**********************

FAIL LONG
2025-08-13 20:00:00
**********************



(12, 5)

In [57]:
from datetime import datetime, timedelta
first_parameter = 0
last_parameter = 30
tp_percentage = 10
sl_percentage=1
total_point = 0

while(last_parameter <= 210):
    end_time = (datetime.now() - timedelta(days=first_parameter)).timestamp() * 1000
    start_time = (datetime.now() - timedelta(days=last_parameter)).timestamp() * 1000
    candles = Candle.objects.filter(symbol__symbol="DOGEUSDT",
                                open_time__gte=start_time,
                                open_time__lte=end_time,
                                interval=Interval.HOUR_4.value[1])
    total, success = strategy(
        candles=candles,
        tp_percentage=tp_percentage,
        sl_percentage=sl_percentage,
    )

    print((total, success))

    fail = total - success
    total_point += (success * tp_percentage) - (fail * sl_percentage)
    first_parameter = last_parameter
    last_parameter += 30


print(total_point)

FAIL LONG
2025-07-16 20:00:00
**********************

FAIL LONG
2025-07-18 04:00:00
**********************

FAIL LONG
2025-07-19 04:00:00
**********************

FAIL LONG
2025-07-20 16:00:00
**********************

SHORT
2025-07-23 08:00:00
**********************

FAIL SHORT
2025-07-25 00:00:00
**********************

FAIL LONG
2025-07-27 20:00:00
**********************

SHORT
2025-07-28 12:00:00
**********************

FAIL LONG
2025-07-31 04:00:00
**********************

SHORT
2025-07-31 16:00:00
**********************

FAIL SHORT
2025-08-02 20:00:00
**********************

FAIL LONG
2025-08-03 08:00:00
**********************

FAIL LONG
2025-08-07 12:00:00
**********************

FAIL LONG
2025-08-07 20:00:00
**********************

FAIL LONG
2025-08-08 04:00:00
**********************

FAIL LONG
2025-08-09 12:00:00
**********************

FAIL LONG
2025-08-12 20:00:00
**********************

FAIL LONG
2025-08-13 04:00:00
**********************

FAIL LONG
2025-08-14 00:00:00
********

In [144]:
Candle.objects.filter(symbol__symbol="ETHUSDT", interval=Interval.MIN_15.value[1]).first().open_time

1725586200000

In [145]:
len(candles)

180

In [8]:
from Database.models import Candle, Symbol

In [9]:
newest_candle = Candle.objects.first()
oldest_candle = Candle.objects.last()

In [10]:
newest_candle.open_time

AttributeError: 'NoneType' object has no attribute 'open_time'

In [11]:
oldest_candle.open_time

AttributeError: 'NoneType' object has no attribute 'open_time'

In [5]:
from ExchangeAPI.APICallManager import CandleAgent
from ExchangeAPI.APICallManager import Interval
agent = CandleAgent(symbol="BTCUSDT", interval=Interval.MIN_15)
candles = agent.fetch_future_candles()
agent.save_to_db(candles)

Saved/updated 100 rows to candles table for BTCUSDT (interval: 900000ms).


0

In [25]:
def check_candles_consistency(symbol_name: str, interval: Interval):
    candles = Candle.objects.filter(symbol__symbol=symbol_name, interval=interval.value[1])
    flag = True
    for previous_candle, candle in zip(candles, candles[1:]):
        if previous_candle.open_time + interval.value[1] != candle.open_time:
            flag = False
            print(previous_candle.open_time)
            break
    if flag:
        print("OKAY")


In [26]:
check_candles_consistency(symbol_name="TRXUSDT", interval=Interval.HOUR_4)

OKAY


In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'HTBot.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [4]:
from Strategies.Indicators import RSI
from Database.models import Candle, Symbol
from ExchangeAPI.APICallManager import Interval
rsi = RSI(
    symbol=Symbol.objects.get(),
    interval=Interval.MIN_15.value[1],
    limit=10000,
)

data = rsi.calculate()


In [5]:
i = 0
for date, rsi in zip(data['open_time'], data['rsi']):
    i += 1
    print(f"{date} -> {rsi}")

print(i)

1744201800000 -> nan
1744202700000 -> nan
1744203600000 -> nan
1744204500000 -> nan
1744205400000 -> nan
1744206300000 -> nan
1744207200000 -> nan
1744208100000 -> nan
1744209000000 -> nan
1744209900000 -> nan
1744210800000 -> nan
1744211700000 -> nan
1744212600000 -> nan
1744213500000 -> nan
1744214400000 -> 65.01668008184868
1744215300000 -> 62.1066326428812
1744216200000 -> 64.57764610310403
1744217100000 -> 68.24446423874465
1744218000000 -> 70.00039655748772
1744218900000 -> 81.38217758745252
1744219800000 -> 81.92779307287735
1744220700000 -> 85.29818827694076
1744221600000 -> 85.53680212064782
1744222500000 -> 86.1919738918066
1744223400000 -> 79.16692016270655
1744224300000 -> 75.36081840444862
1744225200000 -> 77.34414752411209
1744226100000 -> 77.8188949756199
1744227000000 -> 76.61489833776388
1744227900000 -> 76.70629097851076
1744228800000 -> 74.40121610851861
1744229700000 -> 72.87589054559267
1744230600000 -> 76.60447503912391
1744231500000 -> 78.59975872962617
174423240

In [ ]:
1752327000000